<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Метод Эйлера

## Modules – Ordinary Differential Equations
<section class="post-meta">
By Niels Henrik Aase, Thorvald Ballestad and Jon Andreas Støvneng
</section>
Last edited: February 9th 2020

___
Этот блокнот дает более подробное описание теоретических аспектов метода Эйлера, а также предоставляет немного более продвинутый код, чем [реализации метода Эйлера](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/Eulers_method_-_implementation-_done.ipynb). Мы проиллюстрируем, как можно использовать этот простой численный метод для решения дифференциальных уравнений более высокого порядка, а также объясним понятия нестабильности и локальных и глобальных ошибок усечения.
___

## Вступление

Решение дифференциальных уравнений лежит в основе как физики, так и математики. Обыкновенные дифференциальные уравнения, сокращенно ОДУ, появляются во всех видах задач физики, будь то ньютоновская механика (где мы пытаемся найти уравнения движения), теория электромагнитизма или квантовая механика. Однако чаще всего ОДУ не имеют аналитических решений, и нам приходится решать их численно. В этом блокноте мы дадим обширный обзор метода Эйлера, простейшего алгоритма решения ОДУ. В отличие от [простой реализации метода Эйлера](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/Eulers_method_-_implementation-_done.ipynb), мы рассмотрим алгоритм довольно подробно, поскольку мы покажем вывод метода, оценки ошибок и стабильность. Наконец, мы покажем, как мы можем использовать метод Эйлера для решения дифференциальных уравнений более высокого порядка, сводя их к системе дифференциальных уравнений первого порядка. 

Давайте начнем с конкретного примера того, какую проблему мы хотим решить. Рассмотрим дифференциальное уравнение первого порядка 

\begin{equation}
\frac{d}{dt} y(t) = g(y(t), t), \quad(1)
\end{equation}
где $y(t)$ - это функция, которую мы хотим вычислить, а $g(y(t), t)$ - это функция, которая может зависеть от $y(t)$, но она также может иметь явную зависимость от времени. Если у нас есть начальное условие $y(0) = y_0$, как решить (1)? Аналитически существует множество различных схем для решения такого рода уравнений, все со своими собственными ограничениями и областями использования. Примером этого является [Integrating factor](https://en.wikipedia.org/wiki/Integrating_factor), с которым читатель, возможно, уже знаком. Однако все эти аналитические схемы зависят от __существования__ аналитического решения. Решая уравнение (1) численно, мы не ограничиваемся этими видами ОДУ, и, таким образом, мы открыли ящик Пандоры новых ОДУ, которые мы можем решить!

## Теория

### Дискретизация

Одним из недостатков использования численных схем для решения ОДУ является то, что мы должны *дискретизировать* наши временные переменные. Это означает, что переменная времени $t$, которая может принимать только набор предопределенных дискретных значений времени, называемых точками сетки, больше не является непрерывной переменной. Мы определяем этот набор возможных значений времени как


$$
t_n = t_0 + nh, \quad \mathrm{with} \quad n = 0, 1, 2,..., N,
$$

где $t_0$ - это значение времени, при котором мы знаем наше начальное условие, а $h$ - это размер между соседними дискретными значениями времени. Отношение между $N$ и $h$ задается

\begin{equation}
h = \frac{t_N - t_0}{N},\quad(2)
\end{equation}

где $N + 1$ - это количество дискретных временных точек (где + 1 появляется из-за индексации с нуля), в то время как $t_N$ обозначает наибольшее значение времени. Вы можете думать о $h$ как о грубости нашей временной переменной; чем меньше $h$, тем больше точек сетки нам нужно, чтобы охватить один и тот же временной интервал. Например, если мы хотим дискретизировать интервал $[0,1]$ таким образом, чтобы у нас были точки сетки каждые 0.01 секунды, т.е. расстояние между точками сетки равнялось 0.01 (напомним, что это наше определение $h$), мы используем уравнение (2) и получаем $N=100$. В общем, наше численное приближение будет лучше, если мы выберем небольшой шаг $h$. Обратите внимание, что по мере уменьшения размера $h$ увеличивается количество дискретных значений времени между $t_0$ и $t_N$. Мы платим за повышенный уровень точности, увеличивая количество необходимых вычислений, тем самым увеличивая время выполнения нашей программы. Настоятельно рекомендуется поразмышлять над тем, какова необходимая грубость нашей дискретизированной временной переменной, прежде чем решать проблему.

### Метод Эйлера

Существует несколько способов получения метода Эйлера. Здесь мы представляем доказательство, которое основано на рядах Тейлора, поэтому для доказательства требуется базовое исчисление. Любая "хорошая" (мы не уточняем, что для этого требуется) функция $y(t)$ может быть записана как

\begin{equation}
y(t) = \sum_{n=0}^{\infty} \frac{y^{(n)}(t_0)}{n!}(t-t_0)^n, = y(t_0) + \frac{y'(t_0)}{1!}(t-t_0) + \frac{y''(t_0)}{2!}(t-t_0)^2 + ...
\quad(3)
\end{equation}

где $t_0$ - произвольное значение. Мы разложили $y(t)$ в окрестности $t_0$. Теперь мы используем теорему Тейлора и обрываем ряд на первом порядке

\begin{equation}
y(t_0 + h) = y(t_0) + \frac{y'(t_0)}{1!}h + \frac{1}{2!}h^2 y''(\tau),
\quad(4)
\end{equation}

для некоторых $\tau \in [t_0, t_0 + h]$. Комбинируя уравнения (4) и решение для $y'(t_0)$ мы получаем

\begin{equation}
y'(t_0) = \frac{y(t_0 + h)- y(t_0)}{h} + \mathcal{O}(h), 
\quad(5)
\end{equation}

где мы используем [big O-нотацию](https://en.wikipedia.org/wiki/Big_O_notation). Теперь мы используем основную идею метода Эйлера, из (1) мы знаем точное выражение $y'(t_0)$, потому что мы знаем, что $y'(t) = g(y(t), t)$! Подставив его  в (5) и решив относительно $y(t_0 + h)$, мы получим 

\begin{equation}
y(t_0 + h) = y(t_0) + h g(y(t_0), t_0) + \mathcal{O}(h^2).
\quad(6)
\end{equation}

Таким образом, из (6) у нас есть оценка того, какое значение $y$ находится в нашей первой точке сетки $t= t_0 + h$! Выбрав $h$ достаточно маленьким (и надеясь, что слагаемое $ \mathcal{O}$ не слишком велико), получим достаточно точную оценку. Пренебрегая $ \mathcal{O}$, запишем

\begin{equation}
y(t_0 + h) \approx y(t_0) + h g(y(t_0), t_0).
\quad(7)
\end{equation}


Начальное условие в $t_0$ обозначим как $y_0$, и мы можем использовать метод Эйлера, чтобы найти приближение $y$ при $t_1 = t_0 +h$. $y$ в $t_1$ обозначается как $y_1$. Это приближение может быть рассчитано по формуле

$$
y_1 = y_0 + hg(y_0).
$$

Теперь, чтобы найти $y_2$ при $t_2 = t_1 + h = t_0 + 2h$, мы используем ту же формулу, но с $y_1$ вместо $y_0$

$$
y_2 = y_1 + h g(y_1).
$$

Наиболее общая форма метода Эйлера записывается как

\begin{equation}
y_{n+1} = y_n + h g(y_n).
\label{Euler}
\end{equation}

Теперь мы проиллюстрируем, как реализовать метод Эйлера, используя тот же пример, что и в [Implementation of Euler's method notebook](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/Eulers_method_-_implementation-_done.ipynb). 

\begin{equation}
\frac{dy}{dt} = ky(t),
\quad(8)
\end{equation}

где $k=\mathrm{ln}(2)$ и $y(0) = 1$. В предыдущем блокноте мы говорили, что $y(t)$ - это размер популяции бактериальной колонии в момент времени $t$. Мы можем решить уравнение (8) аналитически, получив $y(t) = 2^t$, поэтому нам есть с чем сравнить наши численные результаты.

In [ ]:
# Импорт необходимых библиотек
import numpy as np # NumPy используется для генерации массивов и выполнения некоторых математических операций
import matplotlib.pyplot as plt # Используется для построения графиков результатов

# Updating figure params
"""
newparams = {'figure.figsize': (15, 7), 'axes.grid': False,
             'lines.markersize': 10, 'lines.linewidth': 2,
             'font.size': 15, 'mathtext.fontset': 'stix',
             'font.family': 'STIXGeneral', 'figure.dpi': 200}
plt.rcParams.update(newparams)
""";

In [ ]:
def step_Euler(y, h, f):
    """Выполняет один шаг метода Эйлера.
    
    Параметры:
            y: Численное приближение y в момент времени t
            h: Размер шага
            f: RHS нашего ОДУ (RHS = правая сторона). Может быть любая функция, 
                которая имеет только y в качестве переменной.
    Возвращается:
            next_y: Численное приближение y в момент времени t+h
    """
    next_y = y + h * f(y)
    return next_y


def full_Euler(h, f, y_0 = 1, start_t = 0, end_t = 1):
    """ Полная числовая аппроксимация ОДУ в заданном интервале времени. Выполняет последовательные шаги Эйлера
    с размером шага h от начального времени до времени окончания. Также учитываются начальные значения ОДУ
    
    Параметры:
            h: Размер шага
            f: RHS ОДУ
            y_0 : Начальное условие для y при t = start_t
            start_t : Начальное время, t_0
            end_t : Конец интервала, в котором выполняется метод Эйлера, t_N
    Возвращает:
            y_list: Численное приближение y в моменты времени t_list
            t_list: Равномерно распределенный дискретный список времени с интервалом h. 
                    Время начала = start_t, а время окончания = end_t 
    """
    # Количество шагов дискретизации
    N = int((end_t - start_t) / h)
    # Следуя обозначениям в теории, мы имеем N+1 дискретных значений времени, линейно разнесенных
    t_list = np.linspace(start_t, end_t, N + 1)
    
    # Инициализирует массив для хранения значений y
    y_list = np.zeros(N + 1)
    # Назначит начальное условие первому элементу
    y_list[0] = y_0
    
    # Инициализирует остальную часть массива с помощью N вызовов step_Euler
    for i in range(0, N):
        y_list[i + 1] = step_Euler(y_list[i], h, f)
    return y_list, t_list 

Теперь, когда мы определили наши функции, нам нужно только определить RHS (правую часть) дифференциального уравнения, которое мы в теоретической части обозначили как $g(y)$.

In [ ]:
def g(y):
    """Определяет правую часть нашего дифференциального уравнения. В нашем случае роста бактерий g(y) = k*y
    
    Параметры:
            y: Численное приближение y в момент времени t
        Возвращается:
            скорость роста: текущая численность населения, умноженная на константу пропорциональности.
            В этом случае k = ln(2)
    """
    growth_rate = np.log(2)*y
    return growth_rate

# Теперь мы можем найти численные результаты из метода Эйлера
# и сравнить их с аналитическим решением

# Входные параметры
y_0 = 1  # начальный размер популяции, т. е. одна бактерия
h = 0.01 # Размер шага
t_0 = 0  # Мы определяем время при нашем первоначальном наблюдении как 0
t_N = 10 # через 10 дней после нашего первоначального наблюдения за одной бактерией


# Вычисление результатов по Эйлеру и их построение
y_list, t_list = full_Euler(h, g, y_0, t_0, t_N)
plt.plot(t_list, y_list, label="Numerical", linewidth=1)

# Построение аналитического решения, полученного ранее
plt.plot(t_list,np.power(2, t_list), label="Analytical", linewidth=1)

# Чтобы график выглядел красиво
plt.legend()
plt.title("Размер популяции бактериальной колонии в зависимости от времени")
plt.xlabel(r'$t$ [days]')
plt.ylabel(r'$y$ [# bacteria]')
plt.show()

# Давайте посмотрим, насколько ошибается наша численная аппроксимация через 5 дней.

last_analytical = np.power(2,t_list[-1]) # Извлечение последнего элемента аналитического решения
last_numerical = y_list[-1] # Извлечение последнего элемента численного решения

print("Касательно 10-го дня ошибка составляет: %.2f" %(last_analytical - last_numerical))

Мы видим, что наша модель довольно хорошо работает с размером шага $h=0.01$, поскольку она отклоняется от аналитического решения только на 24 бактериальных клетки, или на 2,4%. Использование меньшего $h$ приведет к меньшей ошибке, и это следующий теоретический аспект, который мы рассмотрим.

### Локальные и глобальные ошибки усечения

В предыдущем разделе теории мы показали следующие соотношения

\begin{equation}
y(t_0 + h) = y(t_0) + h g(y(t_0), t_0) + \mathcal{O}(h^2) \quad (9)
\\
y(t_0 + h) \approx y(t_0) + h g(y(t_0), t_0).
\label{Euler_approx}
\end{equation}

Первое уравнение является __точным__, в то время как другое является приближением. Разложение функции в ряд Тейлора и ее усечение в определенном порядке широко используется в физике (фактически оно является краеугольным камнем почти во всех областях физики). Однако важно изучить последствия этого приближения. 

В численном анализе важным понятием является __локальная__ ошибка усечения. Она описывает ошибку, которую мы совершаем после каждого временного шага. Если у нас есть начальное условие $y_0$, мы можем использовать нашу численную схему, чтобы найти приближение того, каким должен быть $y(t_0 + h) = y_{approx}(t_0+h)$. Если затем мы сравним это приближение с __точным__ решением (т.е. первой строкой (9)), обозначенным $y_{exact}(t_0 + h)$, мы сможем найти локальную ошибку усечения на первом временном шаге, обозначенном $\tau_1$

\begin{equation}
\tau_1 = \mid y_{exact}(t_0 + h) - y_{approx}(t_0 + h) \mid.
\label{local_trunc_error}
\end{equation}

Используя уравнение (9), мы видим, что $\tau_1 = \mathcal{O}(h^2)$. Таким образом, наша локальная ошибка усечения имеет второй порядок, что означает, что если использовать $h$, который составляет только половину исходного $h$, локальная ошибка усечения будет составлять одну четвертую от исходного размера.

Аналогично локальной ошибке усечения, у нас есть __глобальная__ ошибка усечения. На каждом временном шаге нашего численного моделирования мы будем иметь приближение $y(t)$ при $t=t_n$, обозначаемое $y_{approx}(t_n)$. Глобальная ошибка усечения, $e_n$. определяется как

\begin{equation}
e_n = \mid y_{exact}(t_n) - y_{approx}(t_n) \mid.
\label{global_trunc_error}\quad (10)
\end{equation}

Уравнение (10) описывает, насколько далека наша численная схема от точного решения. Обратите внимание, что глобальная ошибка усечения - это не сумма всех локальных ошибок усечения, а скорее накопление ошибок, сделанных на каждом шаге (т.е. Сумма локальных ошибок усечения, __если__ они были определены без абсолютного значения). Здесь мы представляем несколько эвристический подход к поиску $e_n$. Если мы знаем, что наша локальная ошибка усечения имеет порядок $\mathcal{O}(h^2)$, мы знаем, что для каждого временного шага мы получаем ошибку $ah^2$, где $a$ - это просто некоторая константа. Чтобы добраться до $t_N$, нам нужно сделать $N$ шагов, и, используя уравнение (2), мы видим, что количество необходимых шагов обратно пропорционально $h$. Таким образом, накопленная ошибка $e_N = ah^2 \frac{1}{h} = a h$. Следовательно, наш вывод состоит в том, что глобальная ошибка усечения для метода Эйлера имеет порядок $\mathcal{O}(h)$. Обратите внимание, что это соотношение имеет место в целом. Если схема для ОДУ имеет локальную ошибку усечения $\mathcal{O}(h^{p+1})$, глобальная ошибка усечения равна $\mathcal{O}(h^{p})$.

А теперь давайте применим полученную здесь теорию на практике! Следующий код может быть немного более техническим, чем то, что мы делали до сих пор, но он стоит усилий, поскольку мы заново открываем наши теоретические результаты. 

In [ ]:
pip install -U prettytable

In [ ]:
from prettytable import PrettyTable  # импортируется исключительно для получения вывода в хорошем формате


def trunc_errors(f, base=2, h_max_log=-1, h_min_log=-6, y_0=1, start_t=0, end_t=2):
    """Полная численная аппроксимация ОДУ в заданном интервале времени. Выполняет последовательные шаги Эйлера
    с размером шага h от времени начала до времени окончания. Также учитываются начальные значения ОДУ.
    Возвращает как локальную, так и глобальную ошибку усечения для каждого размера шага.

    Параметры:
            f: RHS нашего ОДУ
            base: The base of our logspace, our h_list is created by taking: base **(h_list_log)
            h_min_log: Our smallest element in our h_list is defined by: base**(h_min_log)
            h_max_log: Our largest element in our h_list is defined by: base**(h_max_log)
            y_0 : Initial condition for y at t = start_t
            start_t : The time at the initial condition, t_0
            end_t : The end of the interval where the Euler method is performed, t_N
        Returns:
            t: Table containing the time step size, the global truncation error and the local truncation error
    """
    K = h_max_log - h_min_log + 1
    h_list = np.logspace(h_min_log, h_max_log, K, base=base)  # Creates an array that is evenly spaced on a logscale
    t = PrettyTable(['h', 'Global truncation error', 'Local truncation error'])
    for i in range(len(h_list)):
        y_list, t_list = full_Euler(h_list[i], g, y_0, start_t, end_t)  # Runs Euler Algorithm with a given h
        analytic_list = np.power(2, t_list)
        # Want to format our output nicely, thus we need to add h, Global trunc error
        # and Local trunc error (for the first time step) to our row
        t.add_row([h_list[i], np.abs(y_list[-1] - analytic_list[-1]), np.abs(y_list[1] - analytic_list[1])])
    t.sortby = "h"  # Formatting the table
    t.reversesort = True
    print(t)
    return t


t = trunc_errors(g, 2, h_min_log=-8)



Очевидно, что глобальная ошибка усечения имеет порядок $\mathcal{O}(h)$, поскольку каждый раз, когда мы сокращаем $h$ пополам, соответствующая ошибка усечения также сокращается пополам, в то время как локальная ошибка усечения составляет одну четвертую от предыдущей. 

Порядок глобальной ошибки усечения - это то, что определяет __порядок__ численного метода для решения ОДУ. Мы говорим, что метод Эйлера является методом первого порядка. Существует множество различных способов решения ОДУ, где метод Эйлера является самым простым. В нашем блокно [Движение снаряда](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/projectile_motion.ipynb), мы демонстрируем, как мы можем использовать метод Рунге-Кутты четвертого порядка (у нас также есть записная книжка, в которой описаны более теоретические аспекты [метода Рунге-Кутты](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/runge_kutta_method.ipynb) ).

Для численной моделирования, которое более чувствительно к ошибкам (например, длительное моделирование движения планет), дополнительная работа по реализации метода более высокого порядка стоит вложений, так как количество требуемых временных шагов часто намного меньше.

### Нестабильность

Теперь мы продемонстрируем простой теоретический пример, который иллюстрирует, как метод Эйлера может сломаться, т.е. численное решение начинает резко отклоняться от точного решения. Обычно это происходит, когда численное решение становится большим по величине, в то время как точное решение остается небольшим. 

Давайте посмотрим на ОДУ

$$
\frac{dy}{dt} = -y \quad \mathrm{with} \quad y(0) = 1.
$$

Тривиально, оно имеет точное решение $y(t) = \mathrm{e}^{-t}$. Однако, посмотрим, что происходит с одним шагом в методе Эйлера

\begin{equation}
y_{n+1} = y_n + h g(y_n) = y_n - h y_n = (1-h) y_n.
\end{equation}

Обратите внимание, что для $h=1$ наше решение просто сразу становится нулевым, если $h>1$, наше решение будет колебаться между положительными и отрицательными значениями. Если $h>2$, наше решение будет расти без ограничений (по абсолютному значению), в то время как оно колеблется между положительными и отрицательными значениями. Все три этих случая кардинально отличаются от точного решения! В этом блокноте мы не будем вдаваться в дальнейшие подробности о нестабильности, но мы решили включить этот пример, чтобы продемонстрировать, как численные методы потерпят неудачу при определенных условиях.

### Производные более высокого порядка

Наконец, мы представим, как можно использовать метод Эйлера для решения дифференциальных уравнений более высокого порядка. К сожалению, здесь есть некоторая двусмысленность в отношении номенклатуры. Мы уже говорили о __порядке нашего решателя ОДУ__, но теперь мы введем __порядок дифференциального уравнения__, которое мы хотим решить. Порядок последнего - это просто производная высшего порядка, которая появляется в нашем ОДУ. Например,  

$$
\frac{d^3y}{dt^3} + \frac{dy}{dt} = -y,
$$
отретьего порядка.

#### Пример: Металлический шар, падающий из термосферы

В этом (хорошо сконструированном) примере мы сбросим металлическую сферу из термосферы и изучим ее траекторию к поверхности земли. Верхние слои термосферы находятся в 400 км от поверхности земли. Используя формулу Ньютона для гравитации, мы можем вычислить, что ускорение свободного падения здесь составляет $8.70\, м/с^2$, в то время как на поверхности земли оно составляет $9.82\, м/с^2$. Таким образом, нам нужно обновлять наше значение для силы гравитации, обозначенной $F_G$, в то время как сфера падает.

Нам также необходимо учитывать силу сопротивления $F_D$, действующую на сферу. Давайте напишем второе уравнение Ньютона, чтобы найти уравнения движения

\begin{equation}
ma = m \frac{d^2y}{dt^2} = F_D + F_G = Dv^2 - \frac{GmM}{y^2}.
\label{N2_first}
\end{equation}

Здесь мы обозначаем $D$ как коэффициент сопротивления, $v$ как скорость сферы, $G$ как гравитационную постоянную, а $m$ и $M$ как массу сферы и земли соответственно. Обратите внимание, что ось y направлена в сторону от земли. Слегка перетасовав уравнение, мы находим

\begin{equation}
\frac{d^2y}{dt^2} = \frac{D}{m} v^2 - \frac{GM}{y^2}.
\label{N2_sec}
\end{equation}


Эта форма все еще не совсем готова для использования нашего алгоритма, поэтому мы воспользуемся последним трюком. Мы просто отмечаем $\frac{dy}{dt} = v$ и переписываем уравнение в последний раз как __два__ уравнения.

\begin{equation}
\frac{dy}{dt} = v
\\
\frac{dv}{dt} = \frac{D}{m} v^2 - \frac{GM}{y^2}.
\label{SysODE}
\end{equation}

Мы свели наши дифференциальные уравнения второго порядка к набору из двух уравнений первого порядка! Обратите внимание, что нам все еще нужны два начальных условия: $y_0$ и $v_0$. Применяя метод Эйлера к этим двум уравнениям, мы приходим к

\begin{equation}
y_{n+1} = y_n + h v_n
\\
v_{n+1} = v_n + h \left( \frac{D}{m} v_n^2 - \frac{GM}{y_n^2} \right).
\label{SysODE_Euler} \quad(11)
\end{equation}

Эта методика преобразует дифференциальные уравнений N-го порядка в систему $N$ дифференциальных уравнений первого порядка и является мощным инструментом для решения ОДУ. Теперь мы введем обычную нотацию для решения ОДУ более высокого порядка, введя вектор $\vec{w_n}$, определенный как

$$
\vec{w_n} = \begin{bmatrix}y_n \\ v_n\end{bmatrix}.
$$

Пусть $f$ - функция, которая преобразует наш $w_n$ , как описано уравнением (11),

$$
f(\vec{w_n} ) = f \begin{bmatrix}y_n \\ v_n\end{bmatrix} = \begin{bmatrix}v_n \\ \frac{D}{m} v_n^2 - \frac{GM}{y_n^2}\end{bmatrix}.
$$

$$
\dot{\vec{w_n}}=f(\vec{w_n}). 
$$
Для заинтересованного читателя мы подробнее остановимся на этой теме в блокнотах *[Projectile motion](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/projectile_motion.ipynb)* и *[Runge-Kutta methods](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/runge_kutta_method.ipynb)*.
Реализуя функцию $f$ в качестве правой части нашего ОДУ и слегка изменив наши предыдущие функции, мы можем изучить траекторию нашей сферы.


In [ ]:
## Дифференциальные уравнения высшего порядка

def step_Euler_high(w, t, h, f, deg):
    """Выполняет один шаг метода Эйлера в векторной форме.

    Параметры:
            w: Численное приближение w в момент времени t
            t: Время формирования шага Эйлера
            h: Размер шага
            f: RHS 
        Возвращается:
            next_w: Численное приближение x в момент времени t+h
    """
    next_w = w + h * f(w, t, deg)
    return next_w


# Мы будем хранить данные в матричном виде, поэтому для наглядности проиллюстрируем структуру матрицы здесь

# Матрица M будет иметь следующий вид при полном заполнении данными:
# 
# M[ROW, COLUMN]
# Количество строк равно степени ОДУ, обозначаемой k
# Здесь мы покажем, как будет выглядеть задача, рассмотренная выше
#           N COLUMNS 
#    -----------------------------------
#   | y0   y1   y2   ...  y_N-2   y_N-1
#   | v0   v1   v2   ...  v_N-2   v_N-1
# 
# Запись ":" в M[ROWS, COLUMNS] как например M[:, 0] возвращает массив, содержащий
# первый столбец. M[0, :] возвращает первую строку
#

def full_Euler_high(h, f, init_cond, start_t=0, end_t=1):
    """ Полная численная аппроксимация ОДУ в заданном интервале времени. Выполняет последовательные шаги Эйлера
    с размером шага h от времени начала до времени окончания. Также учитываются начальные значения ОДУ

    Параметры:
            h: Размер шага
            f: RHS нашей ОДЫ (векторная функция)
            init_cond: Массив, содержащий необходимые начальные условия
            start_t : Время в начальном состоянии
            end_t : Конец интервала, в котором выполняется метод Эйлера
        Возвращается:
            M: Матрица с числом строк, равным порядку ОДЫ, и N столбцов
            Содержит численное приближение переменной, которую мы хотим решить в t_list
            t_list: Равномерно распределенный список дискретного времени с интервалом h, 
            время начала = start_t и время окончания = end_t
    """
    
    deg = len(init_cond) # Порядок ОДУ равен количеству необходимых начальных условий
    N = int((end_t - start_t) / h)
    t_list = np.linspace(start_t, end_t, N + 1)
    M = np.zeros((deg, N + 1)) # Матрица, хранящая значения искомой переменной
    # (нулевая производная), а также производные более высокого порядка
    M[:, 0] = init_cond # Сохранение начальных условий
    for i in range(0, N):
        M[:,i + 1] = step_Euler_high(M[:, i], t_list[i], h, f, deg) # Выполнение N шагов Эйлера
    return M, t_list

In [ ]:
D = 0.0025 # Коэффициент лобового сопротивления
m = 1 # масса металлической сферы
M_earth = 5.97 * 10 ** 24 # Масса земли
G = 6.67 * 10 ** (-11) # Гравитационная постоянная


def g(w, t, deg):
    """Определяет правую часть нашего дифференциального уравнения. В нашем случае это векторная функция,
        определяющая уравнение движения.

        Параметры:
                w: Численное приближение w в момент времени t
                t: Время, здесь не имеет значения, поскольку у нас нет явной зависимости от времени
                deg: Степень ОДУ, которую мы хотим решить
            Возвращается:
                next_w: Численное приближение w в момент времени t
        """
    next_w = np.zeros(deg)
    next_w[0] = w[1]
    next_w[1] = D * w[1] ** 2 / m - G * M_earth / w[0] ** 2

    return next_w


In [ ]:
M, t = full_Euler_high(0.01, g, np.array([6771*10**3,0]), 0, 200)

fig = plt.figure(figsize=(18, 6))  

ax1 = plt.subplot(121)
ax1.set_title("Координата падения металлической сферы из термосферы")
ax1.set_xlabel(r"$t$ [s]")
ax1.set_ylabel(r"$y$ [km]")
# Только построение графика первых двадцати секунд траектории и масштабирование y в километры
plt.plot(t[:2000], M[0][:2000] / 10 ** 3)

ax2 = plt.subplot(122)
ax2.set_title("Скорость металлической сферы как функция перемещения во времени")
ax2.set_xlabel(r"$t$ [s]")
ax2.set_ylabel(r"$v_{term}$ [m/s]")
# Построение графика скорости в зависимости от времени в интервале [10с, 20с] для изучения изменения скорости
# после основного ускорения в начале свободного падения
ax2.plot(t[1000:4000], M[1][1000:4000])

fig.tight_layout()
plt.show()

Мы можем считать результаты, полученные методом Эйлера, действительными благодаря нашему физическому пониманию проблемы. На графике справа мы можем изучить скорость металлической сферы, когда она падает на землю. Здесь полезно понятие конечной скорости, т.е. когда сила сопротивления равна силе гравитации. Мы наблюдаем, что после начального ускорения скорость, кажется, выравнивается примерно на 60 м/с. Однако мы знаем, что скорость сферы на самом деле должна увеличиться еще больше, так как гравитационное притяжение земли будет сильнее, когда сфера будет приближаться к центру земли (помните, что гравитация масштабируется как $\frac{1}{r^2}$). Давайте посмотрим, сможем ли мы наблюдать этот эффект в нашем численном решении.

In [ ]:
fig = plt.figure(figsize=(12, 4)) 
plt.plot(t[10000:], M[1][10000:])
plt.title("Скорость металлической сферы в зависимости от времени")
plt.xlabel(r"$t$ [s]")
plt.ylabel(r"$v_{term}$ [m/s]")
plt.show()

Мы ясно наблюдаем эффект возрастающей гравитационной силы! Для получения дополнительных примеров того, как использовать физические принципы для определения достоверности численных результатов (т.е. сохранения энергии и т.д.), Ознакомьтесь с нашей записной книжкой на [Projectile motion](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/projectile_motion.ipynb).

## Вывод

Метод Эйлера - это простая численная процедура для решения обыкновенных дифференциальных уравнений, и его простота проявляется как в реализации, так и в способности метода решать более сложные задачи. Для более сложных задач, которые больше подвержены численным ошибкам, вместо этого следует рассмотреть более мощные методы, такие как метод Рунге-Кутты четвертого порядка (это сделано в вышеупомянутой записной книжке). Однако для простых примеров, которые мы здесь рассмотрели, этого было достаточно!